# Day 16: Permutation Promenade

You come upon a very unusual sight; a group of programs here appear to be dancing.

There are sixteen programs in total, named `a` through `p`.
They start by standing in a line: `a` stands in position 0, `b` stands in position 1, and so on until `p`, which stands in position 15.

The programs' dance consists of a sequence of dance moves:
- Spin, written `sX`, makes X programs move from the end to the front, but maintain their order otherwise. (For example, `s3` on `abcde` produces `cdeab`).
- Exchange, written `xA/B`, makes the programs at positions `A` and `B` swap places.
- Partner, written `pA/B`, makes the programs named `A` and `B` swap places.

In [1]:
def spin(s, X):
    return s[-X:] + s[:-X]


def exchange(s, A, B):
    A, B = min(A, B), max(A, B)
    return s[:A] + s[B] + s[A + 1:B] + s[A] + s[B + 1:]


def partner(s, A, B):
    return exchange(s, s.index(A), s.index(B))


def step(s, cmd):
    if cmd[0] == 's':
        X = int(cmd[1:])
        assert 's%d' % X == cmd
        return spin(s, X)
    elif cmd[0] == 'x':
        A, B = map(int, cmd[1:].split('/'))
        assert 'x%d/%d' % (A, B) == cmd
        return exchange(s, A, B)
    elif cmd[0] == 'p':
        A, B = cmd[1:].split('/')
        assert 'p%s/%s' % (A, B) == cmd
        return partner(s, A, B)
    else:
        raise Exception('Unexpected command %r' % cmd)


assert spin('abcde', 3) == step('abcde', 's3') == 'cdeab'
assert exchange('abcde', 2, 4) == step('abcde', 'x2/4') == 'abedc'
assert partner('abcde', 'e', 'c') == step('abcde', 'pe/c') == 'abedc'

For example, with only five programs standing in a line (`abcde`), they could do the following dance:

In [2]:
example = ['s1', 'x3/4', 'pe/b']
s = 'abcde'

# a spin of size 1
s = step(s, example[0])
assert s == 'eabcd'

# swapping the last two programs
s = step(s, example[1])
assert s == 'eabdc'

# pe/b, swapping programs e and b
s = step(s, example[2])
assert s == 'baedc'

After finishing their dance, the programs end up in order `baedc`.

In [3]:
import string


def run(n, cmds, s=None):
    if s is None:
        s = string.ascii_lowercase[:n]
    for cmd in cmds:
        s = step(s, cmd)
    return s


assert run(5, example) == 'baedc'

You watch the dance for a while and record their dance moves (your puzzle input). In what order are the programs standing after their dance?

In [4]:
puzzle = open('16.input').read().strip().split(',')
run(16, puzzle)

'jkmflcgpdbonihea'

# Part Two

Now that you're starting to get a feel for the dance moves, you turn your attention to the dance as a whole.

Keeping the positions they ended up in from their previous dance, the programs perform it again and again: including the first dance, a total of one billion (`1000000000`) times.

In the example above, their second dance would begin with the order `baedc`, and use the same dance moves:

- `s1`, a spin of size 1: `cbaed`.
- `x3/4`, swapping the last two programs: `cbade`.
- `pe/b`, swapping programs e and b: `ceadb`.

In [5]:
def repeat(n, cmds, r):
    s = None
    for _ in range(r):
        s = run(n, cmds, s)
    return s


assert repeat(5, example, 2) == 'ceadb'

Find a cycle (where we return to the starting point):

In [6]:
def find_cycle(n, cmds):
    start = string.ascii_lowercase[:n]
    s = str(start)
    i = 0
    while True:
        s = run(n, cmds, s)
        i += 1
        if s == start:
            return i


assert find_cycle(5, example) == 4
assert repeat(5, example, 4) == 'abcde'

In what order are the programs standing after their billion dances?

In [7]:
repeat(16, puzzle, 1_000_000_000 % find_cycle(16, puzzle))

'ajcdefghpkblmion'